In [20]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# All of my import declarations
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests


In [21]:
# here we access the webpage and download the content using requests
country_list=requests.get("https://en.wikipedia.org/wiki/ISO_3166-1_numeric")

In [22]:
# Scraping wikipedia for country codes
soup = BeautifulSoup(country_list.text, "html.parser")
rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[1:]
countryTup = [(int(row.findChildren("td")[0].get_text()), row.findChildren("td")[1].get_text()) for row in rows]

In [23]:
# Taking in WVS/EVS survey data
data = pd.read_stata("WVS Data.dta", convert_categoricals=False)
data = data[data.V23 > 0]
data = data[data.V125 > 0]
data = data[data.V126 > 0]
data = data[data.V127 > 0]
data = data[data.V130 > 0]

In [24]:
grouped = data.groupby('V2').mean().reset_index()

In [25]:
# Fixing the headers and adding trust numbers
for value, name in countryTup:
    value = [value]
    grouped.loc[grouped.V2.isin(value),'V2'] = name

trustlist = ['V2', 'V23', 'V125', 'V126', 'V127', 'V130']
trustratings = grouped[trustlist]
trustratings.columns = ['Country', 'PeopleTrust','FamilyTrust', 'NeighborTrust', 'PersonalTrust', 'OtherCountryTrust']


In [26]:
# Taking in economic data and adding it to the trust data
eximdata = pd.read_excel("OECD DITS.xlsx")
eximdata = eximdata[eximdata.TIME == 2013]
collist= ['Reporter country', 'Partner country', 'FLOW', 'Value']
eximdata = eximdata[collist]
joinedset = pd.merge(eximdata, trustratings, how="left", left_on='Reporter country', right_on='Country')
gdpdata = pd.read_excel("GDP.xls", skiprows = 3)
gdpdata.drop(gdpdata.columns[[0,1,2,5]], axis=1, inplace=True)
gdpdata.columns = ['Economy', 'GDP']
joined2 = joinedset.merge(gdpdata, how="left", left_on=['Reporter country'], right_on=['Economy'])
joined2['GDP'] = joined2['GDP'] * 1000000
joined2['Country Trade Percent of GDP'] = joined2['Value'] / (joined2['GDP'])

In [27]:
# Normalizing and cleaning up the data
joined3 = joined2
joined3['PeopleTrust'] = (joined3['PeopleTrust'].max()-joined3['PeopleTrust'])/(joined3['PeopleTrust'].max()-joined3['PeopleTrust'].min())
joined3['FamilyTrust'] = (joined3['FamilyTrust'].max()-joined3['FamilyTrust'])/(joined3['FamilyTrust'].max()-joined3['FamilyTrust'].min())
joined3['NeighborTrust'] = (joined3['NeighborTrust'].max()-joined3['NeighborTrust'])/(joined3['NeighborTrust'].max()-joined3['NeighborTrust'].min())
joined3['PersonalTrust'] = (joined3['PersonalTrust'].max()-joined3['PersonalTrust'])/(joined3['PersonalTrust'].max()-joined3['PersonalTrust'].min())
joined3['OtherCountryTrust'] = (joined3['OtherCountryTrust'].max()-joined3['OtherCountryTrust'])/(joined3['OtherCountryTrust'].max()-joined3['OtherCountryTrust'].min())

EU28 = ['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom']
joined3 = joined3[joined3['Reporter country'].isin(EU28)]
joined3 = joined3[joined3['Partner country'].isin(EU28)]
    
joined3.drop(['Country','Economy'], axis=1)
prelanguage = joined3

In [28]:
# percent able to hold a conversation in English
EU28lang = [('Austria', .64), ('Belgium', .71), ('Bulgaria', .70),\
            ('Cyprus', .75), ('Croatia', .60), ('Czech Republic', .76),\
            ('Denmark', .81), ('Estonia', .64), ('Finland', .67),\
            ('France', .57), ('Germany', .70), ('Greece', .69),\
            ('Hungary', .45), ('Ireland', .89), ('Italy', .69),\
            ('Latvia', .44), ('Lithuania', .54), ('Luxembourg', .63),\
            ('Malta', .80), ('Netherlands', .90), ('Poland', .65),\
            ('Portugal', .62), ('Romania', .50), ('Slovakia', 'Slovak'),\
            ('Slovenia', .68), ('Spain', .58), ('Sweden', .77),\
            ('United Kingdom', .94)]

prelanguage['English Rate'] = None
for name, rate in EU28lang:
    prelanguage.loc[prelanguage['Reporter country'].isin([name]),'English Rate'] = rate
    prelanguage.loc[prelanguage['Partner country'].isin([name]),'PartnerEnglish'] = rate
    
prelanguage['Mutual English'] = prelanguage['English Rate'] * prelanguage['PartnerEnglish']

prebilateral = prelanguage

In [29]:
euroBaromCodes = [(2,'Belgium'),(7,'Denmark'),(4,'Germany'),(11,'Greece'),(5,'Italy'),(12,'Spain'),\
                  (1,'France'),(8,'Ireland'),(10,'United Kingdom'),(6,'Luxembourg'),(3,'Netherlands'),\
                  (13,'Portugal'),(9,'United Kingdom'),(4,'Germany'),(15,'Finland'),(16,'Sweden'),\
                  (17,'Austria')]

In [30]:
# Adding bilateral trust data from 1996
bilateraldata = pd.read_stata("Bilateral Trust.dta", convert_categoricals=False)
questionList = ['v11','v81','v82','v83','v84','v85','v86','v87','v88','v89','v90','v91','v92',\
                'v93','v94','v95','v96','v97','v98','v99','v100','v101','v102','v103']
nameList = ['Country','Austria','Belgium','United Kingdom','Denmark','Netherlands','Finland','France','Germany','Greece',\
            'Ireland','Italy','Luxembourg','Portugal','Spain','Sweden','Switzerland','Norway','Poland','Hungary',\
            'Czech Republic','Slovakia','Russia','Turkey']

bilateraldata = bilateraldata[questionList]
for (value, name) in euroBaromCodes:
    value = [value]
    bilateraldata.loc[bilateraldata.v11.isin(value),'v11'] = name

bilateraldata.columns = nameList

In [31]:
# Cleaning up the finished data
meanbilateraltrust = bilateraldata.groupby('Country').mean().reset_index()
newMerged = prebilateral.merge(meanbilateraltrust, how="left", left_on=['Reporter country'], right_on=['Country'])
for country in nameList[1:]:
    newMerged.loc[newMerged['Partner country'].isin([country]),'Bilateral Trust'] = newMerged[country]

newMerged = newMerged.drop(nameList[1:], axis=1)
newMerged.drop(['Country_x','Economy','Country_y'], axis=1, inplace=True)
newMerged['Bilateral Trust'] = (newMerged['Bilateral Trust'].max() - newMerged['Bilateral Trust'])
newMerged['Bilateral Trust'] = newMerged['Bilateral Trust']/newMerged['Bilateral Trust'].max()
preFDI = newMerged

In [32]:
# Adding FDI Numbers
FDIstats = pd.read_excel('FDI Numbers.xlsx')
all_stats = preFDI.merge(FDIstats, how="left", left_on=['Reporter country'], right_on=['Country'])
all_stats.drop(['Country','Bonds','Commercial bank and other lending'], axis=1, inplace=True)
preoutput = all_stats

In [33]:
# adding partner GDP (for gravity equation)
for country in EU28:
    temp1 = preoutput['GDP'].loc[preoutput['Reporter country'] == country]
    if temp1.empty:
        continue
    else:
        preoutput.loc[preoutput['Partner country'] == country,'PartnerGDP'] = temp1.iloc[0]

pregeo = preoutput

In [34]:
pregeo = preoutput
geodis = pd.read_excel('Economic Distance and Common Law.xlsx', parse_cols='A:F')

pregeo = pregeo.merge(geodis, how="left", on=['Reporter country','Partner country']).drop('Other country', axis=1)
prerel = pregeo


In [35]:
religiondata = pd.read_stata("WVS Data.dta", convert_categoricals=False)
questionList = ['V2','V185']
religiondata = religiondata[questionList]
for value, name in countryTup:
    value = [value]
    religiondata.loc[religiondata.V2.isin(value),'V2'] = name

religiondata.columns = ['Country', 'Religion']
# Categorizing common religions
relTup = [('Catholic/Anglican', 64), ('Muslim', 49), ('Hindu',31), ('Protestant', 62), \
          ('Buddhist', 12), ('Orthodox', 52), ('Jew', 42), ('None', 0), ('Other', 53),\
          ('Catholic/Anglican', 5),('Protestant', 9),('Protestant', 17),('Protestant', 25),\
          ('Muslim', 70), ('Muslim', 75), ('Protestant', 78), ('Native', 4), ('Native', 50),\
          ]

religiondata = religiondata[religiondata.Country != 891]
rellist = [value for (name, value) in relTup]
    
religiondata = religiondata[religiondata.Religion.isin(rellist)]

for name, value in relTup:
    religiondata.loc[religiondata.Religion.isin([value]),'Religion'] = name
                                      
religiondata = religiondata[religiondata['Country'].isin(EU28)]
excelout = religiondata.groupby('Country').Religion.value_counts()
excelout.to_frame(name='Religion').to_excel('Pre-religion.xls')

In [36]:
excelin = pd.read_excel('Post-Religion.xls', parse_cols='B:C,P')
finalData = prerel.merge(excelin, how="left", on=['Reporter country','Partner country'])
output = finalData

In [37]:
output.to_csv('dataset.csv')